In [1]:
import os
import datetime as dt
import pandas as pd
import re


In [2]:
TARGET = """G:\\DCIM\\100GOPRO\\"""

#entries = os.scandir('..')
entries = os.scandir(TARGET)
#entries = os.scandir("""F:\\DCIM\\""")

print(f"Digesting {TARGET}" )

FILE_MODE = 33206
DIR_MODE = 16895

files = []
for e in entries:
    #print(e)
    #print(e.name)
    #print(e.path)
    ctime = dt.datetime.fromtimestamp(e.stat().st_ctime)
    #print(ctime)
    #print()
    files.append({'full_name':e.name, 'path':e.path, 'created':ctime, 'mode':e.stat().st_mode})
    
df = pd.DataFrame(files) 

print(f"Found {len(df)} files")

#make sure everything is either a file or a directory.
def sanity_check(r):
    assert r['mode'] in set([FILE_MODE, DIR_MODE]), f"{r['path']} is unknown filetype {r['mode']}"
    return True

df.apply(sanity_check,axis=1)

df['dir'] = df['mode'] == DIR_MODE
df['file'] = df['mode'] == FILE_MODE

#assume everything is a file for now
assert len(df.loc[df['dir']==True])==0, f"Unexpectedly found a directory in the DCIM folder: {df.loc[df['dir']==True]}"



def digest_filename(r, result=None):
    fn = r['full_name']
    fr = re.compile(r"(?P<name>\w*)\.(?P<extension>\w{3})")  #a2j4l24.txt
    m = fr.search(fn)
    assert m is not None, f"filename {fn} is not in recognized format"
    return m.group(result).lower()

df['extension'] = df.apply(digest_filename, axis=1, result="extension")
df['name'] = df.apply(digest_filename, axis=1, result="name")

#make sure no unexpected extensions
known_extensions = set(['jpg', 'mp4', 'lrv', 'thm', 'wav'])
assert set(df.extension.unique()).issubset(known_extensions), f"unknown extension in {df.extension.unique()}. Expected only {known_extensions}"

print(f"Sanity checks passed")

#throw away unecessary files (lrv, thm)
def destroy_helpers(r):
    os.remove(r.path)
    return True
keep_extensions = set(['jpg', 'mp4'])
print(f"Throwing away any file without extension in {keep_extensions}")
keep_mask = df.extension.isin(keep_extensions)
df.loc[~keep_mask].apply(destroy_helpers, axis=1)
df = df.loc[keep_mask]

print(f"Found {len(df)} videos/photos")

#make sure we understand the filenames
chaptered_video_re = re.compile(r'G(?P<encoding>[H,X])(?P<chapter>\d{2})(?P<file_number>\d{4}).mp4', re.IGNORECASE)
looped_video_re = re.compile(r'G(?P<encoding>[H,X])(?P<loop_prefix>[a-z]{2})(?P<file_number>\d{4}).mp4', re.IGNORECASE)
photo_re = re.compile(r'GOPR(?P<file_number>\d+)\.JPG', re.IGNORECASE)
def check_filename_sanity(r):
    identified = False
    for check in [chaptered_video_re, looped_video_re, photo_re]:
        m = check.search(r.full_name)
        if m is not None:
            identified = True
    assert identified, f"I cannot parse filename {r.full_name}"
    return True
df.apply(check_filename_sanity, axis=1)

print(f"Filenames make sense")

#work out the new name
def new_name(r):  
    date_str = r.created.strftime("%Y-%m-%d_%H%M")
    path = f"{os.path.dirname(r.path)}\\"
    m = chaptered_video_re.search(r.full_name)
    if m is not None:
        return (f"{path}{m.group('file_number')}_{m.group('chapter')}_{date_str}_gopro.mp4".lower())
    else:
        m = photo_re.search(r.full_name)
        if m is not None:
            return(f"{path}{m.group('file_number')}_{date_str}_gopro.jpg".lower())
        else:
            m = looped_video_re.search(r.full_name)
            if m is not None:
                return(f"{path}{m.group('file_number')}_{m.group('loop_prefix')}_{date_str}_gopro.mp4".lower())
            else:
                raise Exception(f"what is {r.full_name}")
                
df['new_name'] = df.apply(new_name, axis=1)

print(f"New names created")
                       
#rename files
def renamer(r):
    try:
        os.rename(r.path, r.new_name)
    except FileNotFoundError:
        raise Exception(f"failed to rename {r.path} to {r.new_name}")
    return True                 
df.apply(renamer, axis=1)
print(f"{len(df)} files renamed")
                       
df

Digesting G:\DCIM\100GOPRO\
Found 15 files
Sanity checks passed
Throwing away any file without extension in {'jpg', 'mp4'}
Found 5 videos/photos
Filenames make sense
New names created
5 files renamed


,created,full_name,mode,path,dir,file,extension,name,new_name
0,2021-07-10 17:30:40,GX010344.MP4,33206,G:\DCIM\100GOPRO\GX010344.MP4,False,True,mp4,gx010344,g:\dcim\100gopro\0344_01_2021-07-10_1730_gopro...
3,2021-07-10 17:37:55,GX010345.MP4,33206,G:\DCIM\100GOPRO\GX010345.MP4,False,True,mp4,gx010345,g:\dcim\100gopro\0345_01_2021-07-10_1737_gopro...
6,2021-07-10 17:41:19,GX010346.MP4,33206,G:\DCIM\100GOPRO\GX010346.MP4,False,True,mp4,gx010346,g:\dcim\100gopro\0346_01_2021-07-10_1741_gopro...
9,2021-07-10 17:48:38,GX010347.MP4,33206,G:\DCIM\100GOPRO\GX010347.MP4,False,True,mp4,gx010347,g:\dcim\100gopro\0347_01_2021-07-10_1748_gopro...
12,2021-07-10 17:59:38,GX010348.MP4,33206,G:\DCIM\100GOPRO\GX010348.MP4,False,True,mp4,gx010348,g:\dcim\100gopro\0348_01_2021-07-10_1759_gopro...


In [5]:
dt.datetime.today()

datetime.datetime(2021, 7, 11, 11, 51, 47, 726057)

In [9]:
try:
    os.mkdir("foo")
except FileExistsError:
    print("already exists")

already exists
